# Legal Assistant Bot

This chatbot retrieves context from a proprietary datasource and the web to answer questions about federal laws in the United States of America (USA).  The proprietary datasource is a CSV file of all federal laws and their revision history in the USA.  The web data required to respond to the user's questions is retrieved using the You.com API.  The chatbot is implemented as an agent in Langchain.

## Install all required packages

In [1]:
%%capture
! pip install langgraph==0.0.59
! pip install pandas==2.2.2
! pip install openai==1.30.3
! pip install langchain==0.2.1
! pip install langchain_community==0.2.1
! pip install langchain_openai==0.1.7
! pip install langchain-anthropic
! pip install langchain_text_splitters==0.2.0
! pip install langchain_core==0.2.1
! pip install numpy==1.26.4
! pip install python-dotenv

## Load in the US Federal Laws dataset and create a vector database representation of this dataset, which will then be converted into a Langchain Retriever and Tool

In [2]:
# Let's take a look at our CSV dataset first
import pandas as pd

# The CSV file can be downloaded from: https://www.nature.com/articles/s41597-023-02758-z#Sec3
df = pd.read_csv("us_laws_dataset.csv")

df.head()

,row_number,action,Title,sal_volume,sal_page_start,BillCitation,congress_number,chapter,session_number,pl_no,date_of_passage,secondary_date,dates_conflict,Source,URL,alternate_sal_volume,alternate_sal_page_start,has_alternate_sal_citation
0,1,An Act,To regulate the time and manner of administeri...,1,23.0,NaN,1,1.0,1.0,NaN,1789-06-01,NaN,NaN,HeinOnline,NaN,NaN,NaN,False
1,2,An Act,"For laying a duty on goods, wares, and merchan...",1,24.0,NaN,1,2.0,1.0,NaN,1789-07-04,NaN,NaN,HeinOnline,NaN,NaN,NaN,False
2,3,An Act,Imposing duties on tonnage.,1,27.0,NaN,1,3.0,1.0,NaN,1789-07-20,NaN,NaN,HeinOnline,NaN,NaN,NaN,False
3,4,An Act,For establishing an executive department to be...,1,28.0,NaN,1,4.0,1.0,NaN,1789-07-27,NaN,NaN,HeinOnline,NaN,NaN,NaN,False
4,5,An Act,To regulate the collection of the duties impos...,1,29.0,NaN,1,5.0,1.0,NaN,1789-07-31,NaN,NaN,HeinOnline,NaN,NaN,NaN,False


In [3]:
import openai
import langchain
import os

In [4]:
#os.environ["YDC_API_KEY"] = "<Insert your YDC API key here>"
#os.environ["OPENAI_API_KEY"] = "<Insert your Open AI API key here>"
#os.environ["ANTHROPIC_API_KEY"] = "<Insert your Anthropic API key here>"

# Or load from .env file
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
from langchain_community.document_loaders.csv_loader import CSVLoader

# The CSV file can be downloaded from: https://www.nature.com/articles/s41597-023-02758-z#Sec3
loader = CSVLoader(file_path = "us_laws_dataset.csv")

data = loader.load()

In [6]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

# split the document into chunks, and vectorize these chunks in a FAISS database
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 100)
docs = text_splitter.split_documents(data)
embeddings = OpenAIEmbeddings()
db = FAISS.from_documents(documents=docs, embedding=embeddings)

In [7]:
# test out the similarity search
query = "What laws and amendments relate to perjury?"
response = db.similarity_search(query, k=10)
# let's look at the first 3 retrieved docs
response[:3]

[Document(page_content='row_number: 1885\naction: An Act\nTitle: In addition to the act, entitled "An act for the prompt settlement of public accounts," and for the punishment of the crime of perjury\nsal_volume: 3\nsal_page_start: 770\nBillCitation: NA\ncongress_number: 17\nchapter: 37\nsession_number: 2\npl_no: NA\ndate_of_passage: 1823-03-01\nsecondary_date: NA\ndates_conflict: NA\nSource: HeinOnline\nURL: NA\nalternate_sal_volume: NA\nalternate_sal_page_start: NA\nhas_alternate_sal_citation: FALSE', metadata={'source': 'us_laws_dataset.csv', 'row': 1884}),
 Document(page_content='row_number: 38724\naction: An Act\nTitle: An act to permit the use of unsworn declarations under penalty of perjury as evidence in Federal proceedings\nsal_volume: 90\nsal_page_start: 2534\nBillCitation: H.R. 15531\ncongress_number: 94\nchapter: NA\nsession_number: 2\npl_no: 94-550\ndate_of_passage: 1976-10-18\nsecondary_date: NA\ndates_conflict: FALSE\nSource: NA\nURL: https://www.govinfo.gov/content/pkg/

In [8]:
from langchain.tools.retriever import create_retriever_tool

# convert this retriver into a tool
db_retriever = db.as_retriever()
db_retriever_tool = create_retriever_tool(
    db_retriever,
    name = "law_dataset_retriever",
    description = "Retrieve relevant context from the US laws dataset."
)

## Instantiating the You.com Tool in Langchain

In [9]:
from langchain_community.tools.you import YouSearchTool
from langchain_community.utilities.you import YouSearchAPIWrapper

api_wrapper = YouSearchAPIWrapper(num_web_results = 10)
ydc_tool = YouSearchTool(api_wrapper=api_wrapper)

In [10]:
# test out the You.com search tool
response = ydc_tool.invoke("Tell me about a recent high-profile case related to antitrust in the USA?")
# let's look at the first 3 results
response[:3]

[Document(page_content='Meijer, Inc. v. Ferring B.V.; Ferring Pharmaceuticals, Inc.; and Aventis Pharmaceuticals [In Re: DDAVP Direct Purchaser Antitrust Litigation] U.S. v. Memphis Board of Realtors', metadata={'url': 'https://www.justice.gov/atr/antitrust-case-filings-alpha', 'thumbnail_url': None, 'title': 'Antitrust Division | Antitrust Case Filings | United States Department of Justice', 'description': 'An official website of the United States government · Official websites use .gov A .gov website belongs to an official government organization in the United States'}),
 Document(page_content="Dentsply International, Inc. v. Antitrust Division of the United States Department of Justice · U.S. v. Freddy Deoliveira · U.S. v. Wilhelm DerMinassian · U.S. v. Eric Descouraux · Leinani Deslandes, Stephanie Turner, et al. v. McDonald's USA, LLC, et al.", metadata={'url': 'https://www.justice.gov/atr/antitrust-case-filings-alpha', 'thumbnail_url': None, 'title': 'Antitrust Division | Antitru

## Instantiate our LLM

In [21]:
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic

llm = ChatAnthropic(model='claude-3-5-sonnet-20240620')
#llm = ChatOpenAI(model="gpt-4o", temperature=0.5)

## Tying it all together

In [12]:
from langgraph.prebuilt import chat_agent_executor
from langgraph.checkpoint import MemorySaver

# Create a checkpointer to use memory
memory = MemorySaver()
# the vector store representation of the CSV dataset and the You.com Search tool will both be passed as tools to the agent
tools = [db_retriever_tool, ydc_tool]
agent_executor = chat_agent_executor.create_tool_calling_executor(llm, tools, checkpointer=memory)

## Let's try it out!

In [13]:
prompt_1 = "What laws in the US pertain to perjury and is there a recent case in the US that relates to a violation of these laws?"

result = agent_executor.invoke(input={"messages": prompt_1}, config={"configurable": {"thread_id": "xyz_789"}})["messages"][-1].content
print(result)

Based on the information retrieved, I can provide you with an overview of the laws pertaining to perjury in the United States and a recent case related to a violation of these laws.

Laws pertaining to perjury in the US:

1. The main federal statutes criminalizing perjury are 18 U.S.C. §§ 1621 and 1623.

2. 18 U.S.C. § 1621 (Perjury generally):
   - This is the traditional, broadly applicable perjury statute.
   - It applies to false statements made under oath before legislative, administrative, or judicial bodies.
   - Key elements include:
     a) Taking an oath before a competent tribunal, officer, or person.
     b) Willfully stating or subscribing to any material matter which the person does not believe to be true.
   - The penalty includes fines and imprisonment for up to five years.

3. 18 U.S.C. § 1623 (False declarations before grand jury or court):
   - This statute specifically addresses false statements made in court or before a grand jury.
   - It was added in 1971 as a re

In [14]:
prompt_2 = "What is the most famous US Supreme Court perjury case?"
result = agent_executor.invoke(input={"messages": prompt_2}, config={"configurable": {"thread_id": "xyz_789"}})["messages"][-1].content
print(result)

Based on the search results, the most famous US Supreme Court perjury case is Bronston v. United States, 409 U.S. 352 (1973). This case is considered a seminal decision in US perjury law. Here are the key points about this case:

1. Significance: Bronston v. United States is the controlling legal standard for perjury in federal jurisprudence.

2. Decision: The Supreme Court strictly construed the federal perjury statute.

3. Ruling: Chief Justice Warren Burger, writing for a unanimous Court, held that responses to questions made under oath that relay truthful information in themselves but are intended to mislead or evade the examiner could not be prosecuted as perjury.

4. Implication: The criminal justice system must rely on more carefully worded follow-up questions to prevent evasive answers, rather than prosecuting for perjury.

5. Legal Standard: The Court established that for a statement to be considered perjury, it must be false, concern a material matter, and be made with the wi

In [15]:
prompt_3 =  "Based on your knowledge of all laws in the US pertaining to perjury, were there any other laws that could have been applied in this case?"
result = agent_executor.invoke(input={"messages":prompt_3}, config={"configurable": {"thread_id": "xyz_789"}})["messages"][-1].content
print(result)

Based on the information provided about US laws pertaining to perjury and considering the Bronston v. United States case, it's important to note that the laws have evolved over time. However, to answer your question about whether other laws could have been applied in this case, we need to consider the laws that were in effect at the time of the Bronston case in 1973. Let's analyze the situation:

1. Main Perjury Statute: The primary law applied in Bronston v. United States was likely 18 U.S.C. § 1621, which is the general federal perjury statute. This law was already in place at the time of the case.

2. False Declarations Statute: In 1970, just a few years before the Bronston case, Congress enacted 18 U.S.C. § 1623, which specifically addresses false declarations before a grand jury or court. This law could potentially have been applied, but it was relatively new at the time of the Bronston case.

3. Unsworn Declarations: The act "to permit the use of unsworn declarations under penalt

## Let's create a Python class that encapsulates the code above.  This will enable users to easily create chatbots for new use cases with custom CSV datasets

In [22]:
import secrets
from typing import Union

class CSV_QA_Bot:
    def __init__(self, llm: Union[ChatOpenAI, ChatAnthropic], csv_files: list[str], num_web_results_to_fetch: int = 10):
        self._llm = llm
        
        docs = self._load_csv_files(csv_files)
        
        # split the docs into chunks, vectorize the chunks and load them into a vector store
        db = self._create_vector_store(docs)
        
        # create a retriever from the vector store
        self._faiss_retriever = db.as_retriever()
        
        # convert this retriever into a Langchain tool
        self._faiss_retriever_tool = create_retriever_tool(
            self._faiss_retriever,
            name = "law_dataset_retriever",
            description = "Retrieve relevant context from the US laws dataset."
        )
        
        # instantiate the YDC search tool in Langchain
        self._ydc_api_wrapper = YouSearchAPIWrapper(num_web_results=num_web_results_to_fetch)
        self._ydc_search_tool = YouSearchTool(api_wrapper=self._ydc_api_wrapper)
        
        
        # create a list of tools that will be supplied to the Langchain agent
        self._tools = [self._faiss_retriever_tool, self._ydc_search_tool]
        
        # Create a checkpointer to use memory
        self._memory = MemorySaver()
        
        # create the agent executor
        self._agent_executor = chat_agent_executor.create_tool_calling_executor(self._llm, tools, checkpointer=memory)
        
        # generate a thread ID for to keep track of conversation history
        self._thread_id = self._generate_thread_id()

    def _load_csv_files(self, csv_files: list[str]) -> list:
        docs = []
        for file in csv_files:
            data_loader = CSVLoader(file)
            docs.extend(data_loader.load())
        return docs
    
    def _create_vector_store(self, docs: list) -> FAISS:
        text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 100)
        chunked_docs = text_splitter.split_documents(docs)
        embeddings = OpenAIEmbeddings()
        return FAISS.from_documents(documents=chunked_docs, embedding=embeddings)

    def _generate_thread_id(self) -> str:
        thread_id = secrets.token_urlsafe(16)
        return thread_id
    
    def invoke_bot(self, input_str: str) -> str:
        input = {"messages": input_str}
        config = {"configurable": {"thread_id": self._thread_id}}
        output = self._agent_executor.invoke(input=input, config=config)["messages"][-1].content
        return output

## Let's try it out!

In [23]:
conversational_agent = CSV_QA_Bot(llm, csv_files=["us_laws_dataset.csv"])

In [24]:
prompt_1 = "What laws in the US pertain to perjury and is there a recent case in the US that relates to a violation of these laws?"

print(conversational_agent.invoke_bot(prompt_1))

Based on the search results, I can provide information about a recent perjury case in the United States. The case involves Craig German, a 60-year-old man from Kernersville, North Carolina. This case is particularly interesting because it relates to perjury committed during the sentencing phase of a previous case.

Here are the key details of this recent perjury case:

1. Background: Craig German was previously convicted for conspiring to steal trade secrets from aircraft manufacturing companies.

2. New Charges: After his initial conviction, German faced additional charges for committing perjury in his prior case and for providing false statements to a government agency (the FBI).

3. Trial and Conviction: A federal jury in the U.S. District Court for the Southern District of Georgia found German guilty of perjury and false statements to a government agency after a three-day trial.

4. Specifics of the Perjury:
   - During the sentencing portion of his prior case, German testified und

In [25]:
prompt_2 = "What is the most famous US Supreme Court perjury case?"

print(conversational_agent.invoke_bot(prompt_2))

Based on the search results, the most famous US Supreme Court perjury case is Bronston v. United States, 409 U.S. 352 (1973). This case is considered seminal in US jurisprudence regarding perjury. Here are the key points about this landmark case:

1. Significance: Bronston v. United States is the controlling legal standard for perjury in federal jurisprudence and has been widely cited since its decision.

2. Ruling: The Supreme Court, in a unanimous decision written by Chief Justice Warren Burger, strictly construed the federal perjury statute.

3. Key principle: The Court held that responses to questions made under oath that relay truthful information in themselves but are intended to mislead or evade the examiner cannot be prosecuted as perjury.

4. Implications: This decision essentially created a loophole in perjury statutes, allowing witnesses to potentially mislead without legal consequences as long as their statements are literally true.

5. Remedy: The Court stated that the cri

In [26]:
prompt_3 =  "Based on your knowledge of all laws in the US pertaining to perjury, were there any other laws that could have been applied in this case?"

print(conversational_agent.invoke_bot(prompt_3))

Based on the information retrieved about US laws pertaining to perjury and considering the Bronston v. United States case, it's important to note that the Bronston case was decided in 1973, which predates some of the laws mentioned in the retrieval. However, we can analyze whether any of these laws or other existing laws at the time could have been applied in this case:

1. The 1823 Act for the punishment of the crime of perjury: This act was likely superseded by more modern statutes by the time of the Bronston case, but it shows the long-standing nature of perjury laws in the US.

2. The federal perjury statute (18 U.S.C. § 1621): This was the primary law under consideration in the Bronston case. The Court's interpretation of this statute led to the ruling that literal truth, even if misleading, cannot be prosecuted as perjury.

3. False Statements Statute (18 U.S.C. § 1001): While not specifically a perjury statute, this law prohibits making false statements to federal officials. It 